In [1]:
from collections import OrderedDict
from robustness import datasets
from robustness.tools.breeds_helpers import make_living17
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import sampler
import os
import torchvision
import pandas as pd
# for plotting

from trainer_functions.customImageNettrainer import build_resnet50, evaluate_customImageNet, train_customImageNet

In [2]:
data_dir = '/Users/rada/.mxnet/datasets/imagenet'
info_dir = '/Users/rada/Documents/GitHub/BREEDS-Benchmarks/imagenet_class_hierarchy/modified'

n_subset = 850
lr = 0.0005

In [3]:
""" # empty dataframe
df = pd.DataFrame(columns=['accuracy', "lr", "state", "n_subset"])

ret = make_living17(info_dir, split="rand")
superclasses, subclass_split, label_map = ret
train_subclasses, test_subclasses = subclass_split

dataset_source = datasets.CustomImageNet(data_dir, train_subclasses)
loaders_source = dataset_source.make_loaders(
    workers=10, batch_size=64, shuffle_train=64, shuffle_val=64, subset=850)
train_loader_source, val_loader_source = loaders_source

dataset_target = datasets.CustomImageNet(data_dir, test_subclasses)
loaders_target = dataset_target.make_loaders(
    workers=10, batch_size=64, shuffle_train=True, shuffle_val=True, subset=850)
train_loader_target, val_loader_target = loaders_target

device = torch.device("mps")
tune_net = build_resnet50(device)

optimizer = optim.Adam(tune_net.fc.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
acc = train_customImageNet(tune_net, train_loader_source,
                           val_loader_source, optimizer, scheduler, device=device, epochs=3)

optimizer = optim.Adam(tune_net.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
acc = train_customImageNet(tune_net, train_loader_source,
                           val_loader_source, optimizer, scheduler, device=device, epochs=2)

# first layer is fine-tuned
optimizer = optim.Adam(tune_net.layer1.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
acc = train_customImageNet(tune_net, train_loader_target,
                           val_loader_target, optimizer, scheduler, device=device, epochs=15)

# adding results to dataframe
dict = {}
dict['accuracy'] = acc
dict['lr'] = lr
dict['state'] = "first layer"  # change when run
dict['n_subset'] = n_subset
df_temp = pd.DataFrame(dict, index=[0])
df = pd.concat([df, df_temp])

# change when run
df.to_csv("Living17_layer1.csv") """


' # empty dataframe\ndf = pd.DataFrame(columns=[\'accuracy\', "lr", "state", "n_subset"])\n\nret = make_living17(info_dir, split="rand")\nsuperclasses, subclass_split, label_map = ret\ntrain_subclasses, test_subclasses = subclass_split\n\ndataset_source = datasets.CustomImageNet(data_dir, train_subclasses)\nloaders_source = dataset_source.make_loaders(\n    workers=10, batch_size=64, shuffle_train=64, shuffle_val=64, subset=850)\ntrain_loader_source, val_loader_source = loaders_source\n\ndataset_target = datasets.CustomImageNet(data_dir, test_subclasses)\nloaders_target = dataset_target.make_loaders(\n    workers=10, batch_size=64, shuffle_train=True, shuffle_val=True, subset=850)\ntrain_loader_target, val_loader_target = loaders_target\n\ndevice = torch.device("mps")\ntune_net = build_resnet50(device)\n\noptimizer = optim.Adam(tune_net.fc.parameters(), lr=0.0001)\nscheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)\nacc = train_customImageNet(tune_net, train_load

In [4]:
# empty dataframe
df = pd.DataFrame(columns=['accuracy', "lr", "state", "n_subset"])

ret = make_living17(info_dir, split="rand")
superclasses, subclass_split, label_map = ret
train_subclasses, test_subclasses = subclass_split

dataset_source = datasets.CustomImageNet(data_dir, train_subclasses)
loaders_source = dataset_source.make_loaders(
    workers=10, batch_size=64, shuffle_train=64, shuffle_val=64, subset=850)
train_loader_source, val_loader_source = loaders_source

dataset_target = datasets.CustomImageNet(data_dir, test_subclasses)
loaders_target = dataset_target.make_loaders(
    workers=10, batch_size=64, shuffle_train=True, shuffle_val=True, subset=850)
train_loader_target, val_loader_target = loaders_target

device = torch.device("mps")
tune_net = build_resnet50(device)

optimizer = optim.Adam(tune_net.fc.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
acc = train_customImageNet(tune_net, train_loader_source,
                           val_loader_source, optimizer, scheduler, device=device, epochs=3)

optimizer = optim.Adam(tune_net.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
acc = train_customImageNet(tune_net, train_loader_source,
                           val_loader_source, optimizer, scheduler, device=device, epochs=2)

# second layer is fine-tuned
optimizer = optim.Adam(tune_net.layer2.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
acc = train_customImageNet(tune_net, train_loader_target,
                           val_loader_target, optimizer, scheduler, device=device, epochs=15)

# adding results to dataframe
dict = {}
dict['accuracy'] = acc
dict['lr'] = lr
dict['state'] = "second layer"  # change when run
dict['n_subset'] = n_subset
df_temp = pd.DataFrame(dict, index=[0])
df = pd.concat([df, df_temp])

# change when run
df.to_csv("Living17_layer2.csv")

==> Preparing dataset custom_imagenet..
==> Preparing dataset custom_imagenet..
Building model...
Epoch 0, loss = 4.1121, lr 0.00100000
Got 9 / 1700 correct (0.53)

Epoch 1, loss = 3.1633, lr 0.00100000
Got 116 / 1700 correct (6.82)

Epoch 2, loss = 2.3018, lr 0.00100000
Got 325 / 1700 correct (19.12)

Epoch 0, loss = 1.1397, lr 0.00100000
Got 1147 / 1700 correct (67.47)

Epoch 1, loss = 0.8010, lr 0.00100000
Got 1018 / 1700 correct (59.88)

Epoch 0, loss = 2.4053, lr 0.00050000
Got 829 / 1700 correct (48.76)

Epoch 1, loss = 2.7847, lr 0.00050000
Got 907 / 1700 correct (53.35)

Epoch 2, loss = 1.7788, lr 0.00050000
Got 969 / 1700 correct (57.00)

Epoch 3, loss = 1.9061, lr 0.00050000
Got 1003 / 1700 correct (59.00)

Epoch 4, loss = 1.8682, lr 0.00050000
Got 1028 / 1700 correct (60.47)

Epoch 5, loss = 1.0223, lr 0.00050000
Got 1046 / 1700 correct (61.53)

Epoch 6, loss = 0.7554, lr 0.00050000
Got 1082 / 1700 correct (63.65)

Epoch 7, loss = 1.3301, lr 0.00050000
Got 1104 / 1700 correc

In [5]:
# empty dataframe
df = pd.DataFrame(columns=['accuracy', "lr", "state", "n_subset"])

ret = make_living17(info_dir, split="rand")
superclasses, subclass_split, label_map = ret
train_subclasses, test_subclasses = subclass_split

dataset_source = datasets.CustomImageNet(data_dir, train_subclasses)
loaders_source = dataset_source.make_loaders(
    workers=10, batch_size=64, shuffle_train=64, shuffle_val=64, subset=850)
train_loader_source, val_loader_source = loaders_source

dataset_target = datasets.CustomImageNet(data_dir, test_subclasses)
loaders_target = dataset_target.make_loaders(
    workers=10, batch_size=64, shuffle_train=True, shuffle_val=True, subset=850)
train_loader_target, val_loader_target = loaders_target

device = torch.device("mps")
tune_net = build_resnet50(device)

optimizer = optim.Adam(tune_net.fc.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
acc = train_customImageNet(tune_net, train_loader_source,
                           val_loader_source, optimizer, scheduler, device=device, epochs=3)

optimizer = optim.Adam(tune_net.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
acc = train_customImageNet(tune_net, train_loader_source,
                           val_loader_source, optimizer, scheduler, device=device, epochs=2)

# third layer is fine-tuned
optimizer = optim.Adam(tune_net.layer3.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
acc = train_customImageNet(tune_net, train_loader_target,
                           val_loader_target, optimizer, scheduler, device=device, epochs=15)

# adding results to dataframe
dict = {}
dict['accuracy'] = acc
dict['lr'] = lr
dict['state'] = "third layer"  # change when run
dict['n_subset'] = n_subset
df_temp = pd.DataFrame(dict, index=[0])
df = pd.concat([df, df_temp])

# change when run
df.to_csv("Living17_layer3.csv")

==> Preparing dataset custom_imagenet..
==> Preparing dataset custom_imagenet..
Building model...
Epoch 0, loss = 5.1348, lr 0.00100000
Got 9 / 1700 correct (0.53)

Epoch 1, loss = 3.0784, lr 0.00100000
Got 104 / 1700 correct (6.12)

Epoch 2, loss = 2.5171, lr 0.00100000
Got 313 / 1700 correct (18.41)

Epoch 0, loss = 0.8862, lr 0.00100000
Got 736 / 1700 correct (43.29)

Epoch 1, loss = 1.1265, lr 0.00100000
Got 890 / 1700 correct (52.35)

Epoch 0, loss = 1.8971, lr 0.00050000
Got 1089 / 1700 correct (64.06)

Epoch 1, loss = 1.4336, lr 0.00050000
Got 1220 / 1700 correct (71.76)

Epoch 2, loss = 0.5061, lr 0.00050000
Got 1293 / 1700 correct (76.06)

Epoch 3, loss = 0.9557, lr 0.00050000
Got 1335 / 1700 correct (78.53)

Epoch 4, loss = 0.8969, lr 0.00050000
Got 1357 / 1700 correct (79.82)

Epoch 5, loss = 0.6619, lr 0.00050000
Got 1370 / 1700 correct (80.59)

Epoch 6, loss = 0.3786, lr 0.00050000
Got 1352 / 1700 correct (79.53)

Epoch 7, loss = 0.1445, lr 0.00050000
Got 1340 / 1700 corre

In [6]:
# empty dataframe
df = pd.DataFrame(columns=['accuracy', "lr", "state", "n_subset"])

ret = make_living17(info_dir, split="rand")
superclasses, subclass_split, label_map = ret
train_subclasses, test_subclasses = subclass_split

dataset_source = datasets.CustomImageNet(data_dir, train_subclasses)
loaders_source = dataset_source.make_loaders(
    workers=10, batch_size=64, shuffle_train=64, shuffle_val=64, subset=850)
train_loader_source, val_loader_source = loaders_source

dataset_target = datasets.CustomImageNet(data_dir, test_subclasses)
loaders_target = dataset_target.make_loaders(
    workers=10, batch_size=64, shuffle_train=True, shuffle_val=True, subset=850)
train_loader_target, val_loader_target = loaders_target

device = torch.device("mps")
tune_net = build_resnet50(device)

optimizer = optim.Adam(tune_net.fc.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
acc = train_customImageNet(tune_net, train_loader_source,
                           val_loader_source, optimizer, scheduler, device=device, epochs=3)

optimizer = optim.Adam(tune_net.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
acc = train_customImageNet(tune_net, train_loader_source,
                           val_loader_source, optimizer, scheduler, device=device, epochs=2)

# fourth layer is fine-tuned
optimizer = optim.Adam(tune_net.layer4.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
acc = train_customImageNet(tune_net, train_loader_target,
                           val_loader_target, optimizer, scheduler, device=device, epochs=15)

# adding results to dataframe
dict = {}
dict['accuracy'] = acc
dict['lr'] = lr
dict['state'] = "fourth layer"  # change when run
dict['n_subset'] = n_subset
df_temp = pd.DataFrame(dict, index=[0])
df = pd.concat([df, df_temp])

# change when run
df.to_csv("Living17_layer4.csv")

==> Preparing dataset custom_imagenet..
==> Preparing dataset custom_imagenet..
Building model...
Epoch 0, loss = 4.4758, lr 0.00100000
Got 8 / 1700 correct (0.47)

Epoch 1, loss = 3.4250, lr 0.00100000
Got 122 / 1700 correct (7.18)

Epoch 2, loss = 2.4645, lr 0.00100000
Got 312 / 1700 correct (18.35)

Epoch 0, loss = 1.5637, lr 0.00100000
Got 972 / 1700 correct (57.18)

Epoch 1, loss = 0.3767, lr 0.00100000
Got 1013 / 1700 correct (59.59)

Epoch 0, loss = 1.3796, lr 0.00050000
Got 1288 / 1700 correct (75.76)

Epoch 1, loss = 0.8569, lr 0.00050000
Got 1364 / 1700 correct (80.24)

Epoch 2, loss = 0.9448, lr 0.00050000
Got 1420 / 1700 correct (83.53)

Epoch 3, loss = 0.3999, lr 0.00050000
Got 1451 / 1700 correct (85.35)

Epoch 4, loss = 1.1652, lr 0.00050000
Got 1431 / 1700 correct (84.18)

Epoch 5, loss = 0.8514, lr 0.00050000
Got 1436 / 1700 correct (84.47)

Epoch 6, loss = 0.2608, lr 0.00050000
Got 1461 / 1700 correct (85.94)

Epoch 7, loss = 0.3002, lr 0.00050000
Got 1444 / 1700 corr

In [7]:
# empty dataframe
df = pd.DataFrame(columns=['accuracy', "lr", "state", "n_subset"])

ret = make_living17(info_dir, split="rand")
superclasses, subclass_split, label_map = ret
train_subclasses, test_subclasses = subclass_split

dataset_source = datasets.CustomImageNet(data_dir, train_subclasses)
loaders_source = dataset_source.make_loaders(
    workers=10, batch_size=64, shuffle_train=64, shuffle_val=64, subset=850)
train_loader_source, val_loader_source = loaders_source

dataset_target = datasets.CustomImageNet(data_dir, test_subclasses)
loaders_target = dataset_target.make_loaders(
    workers=10, batch_size=64, shuffle_train=True, shuffle_val=True, subset=850)
train_loader_target, val_loader_target = loaders_target

device = torch.device("mps")
tune_net = build_resnet50(device)

optimizer = optim.Adam(tune_net.fc.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
acc = train_customImageNet(tune_net, train_loader_source,
                           val_loader_source, optimizer, scheduler, device=device, epochs=3)

optimizer = optim.Adam(tune_net.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
acc = train_customImageNet(tune_net, train_loader_source,
                           val_loader_source, optimizer, scheduler, device=device, epochs=2)

# last layer is fine-tuned
optimizer = optim.Adam(tune_net.fc.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
acc = train_customImageNet(tune_net, train_loader_target,
                           val_loader_target, optimizer, scheduler, device=device, epochs=15)

# adding results to dataframe
dict = {}
dict['accuracy'] = acc
dict['lr'] = lr
dict['state'] = "last layer"  # change when run
dict['n_subset'] = n_subset
df_temp = pd.DataFrame(dict, index=[0])
df = pd.concat([df, df_temp])

# change when run
df.to_csv("Living17_last_layer.csv")

==> Preparing dataset custom_imagenet..
==> Preparing dataset custom_imagenet..
Building model...
Epoch 0, loss = 4.5696, lr 0.00100000
Got 8 / 1700 correct (0.47)

Epoch 1, loss = 3.2621, lr 0.00100000
Got 110 / 1700 correct (6.47)

Epoch 2, loss = 2.3200, lr 0.00100000
Got 325 / 1700 correct (19.12)

Epoch 0, loss = 1.6083, lr 0.00100000
Got 927 / 1700 correct (54.53)

Epoch 1, loss = 0.8425, lr 0.00100000
Got 1132 / 1700 correct (66.59)

Epoch 0, loss = 1.5924, lr 0.00050000
Got 867 / 1700 correct (51.00)

Epoch 1, loss = 1.5459, lr 0.00050000
Got 955 / 1700 correct (56.18)

Epoch 2, loss = 1.5987, lr 0.00050000
Got 1028 / 1700 correct (60.47)

Epoch 3, loss = 0.9419, lr 0.00050000
Got 1099 / 1700 correct (64.65)

Epoch 4, loss = 1.1386, lr 0.00050000
Got 1142 / 1700 correct (67.18)

Epoch 5, loss = 1.0972, lr 0.00050000
Got 1174 / 1700 correct (69.06)

Epoch 6, loss = 1.8561, lr 0.00050000
Got 1205 / 1700 correct (70.88)

Epoch 7, loss = 0.8144, lr 0.00050000
Got 1236 / 1700 correc

In [8]:
# empty dataframe
df = pd.DataFrame(columns=['accuracy', "lr", "state", "n_subset"])

ret = make_living17(info_dir, split="rand")
superclasses, subclass_split, label_map = ret
train_subclasses, test_subclasses = subclass_split

dataset_source = datasets.CustomImageNet(data_dir, train_subclasses)
loaders_source = dataset_source.make_loaders(
    workers=10, batch_size=64, shuffle_train=64, shuffle_val=64, subset=850)
train_loader_source, val_loader_source = loaders_source

dataset_target = datasets.CustomImageNet(data_dir, test_subclasses)
loaders_target = dataset_target.make_loaders(
    workers=10, batch_size=64, shuffle_train=True, shuffle_val=True, subset=850)
train_loader_target, val_loader_target = loaders_target

device = torch.device("mps")
tune_net = build_resnet50(device)

optimizer = optim.Adam(tune_net.fc.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
acc = train_customImageNet(tune_net, train_loader_source,
                           val_loader_source, optimizer, scheduler, device=device, epochs=3)

optimizer = optim.Adam(tune_net.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
acc = train_customImageNet(tune_net, train_loader_source,
                           val_loader_source, optimizer, scheduler, device=device, epochs=2)

# all layers is fine-tuned
optimizer = optim.Adam(tune_net.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
acc = train_customImageNet(tune_net, train_loader_target,
                           val_loader_target, optimizer, scheduler, device=device, epochs=15)

# adding results to dataframe
dict = {}
dict['accuracy'] = acc
dict['lr'] = lr
dict['state'] = "all layers"  # change when run
dict['n_subset'] = n_subset
df_temp = pd.DataFrame(dict, index=[0])
df = pd.concat([df, df_temp])

# change when run
df.to_csv("Living17_alllayers.csv")

==> Preparing dataset custom_imagenet..
==> Preparing dataset custom_imagenet..
Building model...
Epoch 0, loss = 4.4790, lr 0.00100000
Got 14 / 1700 correct (0.82)

Epoch 1, loss = 2.7285, lr 0.00100000
Got 134 / 1700 correct (7.88)

Epoch 2, loss = 2.2932, lr 0.00100000
Got 319 / 1700 correct (18.76)

Epoch 0, loss = 1.2226, lr 0.00100000
Got 869 / 1700 correct (51.12)

Epoch 1, loss = 1.5237, lr 0.00100000
Got 959 / 1700 correct (56.41)

Epoch 0, loss = 1.5870, lr 0.00050000
Got 1182 / 1700 correct (69.53)

Epoch 1, loss = 0.6880, lr 0.00050000
Got 1383 / 1700 correct (81.35)

Epoch 2, loss = 0.8356, lr 0.00050000
Got 1410 / 1700 correct (82.94)

Epoch 3, loss = 0.7548, lr 0.00050000
Got 1385 / 1700 correct (81.47)

Epoch 4, loss = 0.1644, lr 0.00050000
Got 1423 / 1700 correct (83.71)

Epoch 5, loss = 0.3517, lr 0.00050000
Got 1389 / 1700 correct (81.71)

Epoch 6, loss = 0.7651, lr 0.00050000
Got 1415 / 1700 correct (83.24)

Epoch 7, loss = 0.4718, lr 0.00050000
Got 1414 / 1700 corr